In [1]:
import sqlite3
import pandas as pd
import numpy as np
from langdetect import DetectorFactory
from langdetect import detect
from zipfile import ZipFile
from collections import Counter

In [2]:
db_path = '../Data/input/schema_org_reviews.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [3]:
df = pd.read_sql_query("SELECT * from reviews", conn)
df.head()

,NODE,URL,REVIEWBODY,REVIEWRATING,ITEMREVIEWED,REVIEWASPECT,REVIEWAUTHOR,REVIEWDESCR,BESTRATING,WORSTRATING
0,_:node2c2e3ffd705962c891db9ab89a44cacc,http://20te20.ir/527007-%D8%AA%D9%86%D9%88%D8%...,None,<http://20te20.ir/527007-%D8%AA%D9%86%D9%88%D8...,None,None,None,None,None,None
1,_:node1f865f4bfa138ac184ddbbd383299621,http://67kmm.com/~hello-neighbor/android,"""Hello Neighbor est un jeu d'horreur \u00C3\u0...",None,None,None,None,None,None,None
2,_:node7d8efbe93e9573eb609b41cfa3be84,http://67kmm.com/~snapchat/android,"""Bon7Snapchat est une application de partage i...",None,None,None,None,None,None,None
3,_:node98788ddf61c5e4913d4ee048868b02,http://20te20.ir/517230-%D8%A7%D8%AA%D9%88-%D8...,None,<http://20te20.ir/517230-%D8%A7%D8%AA%D9%88-%D...,None,None,None,None,None,None
4,_:nodebb685880f9d030fca1b194c14548d8b5,http://67kmm.com/~grand-theft-auto-san-andreas...,"""Par Matt Mullarkey-TonerExcellent9Du haut de ...",None,None,None,_:node55c1dea05a3a6929edebabe6265ba3b,None,None,None


In [4]:
# get null out for where it is important
df.dropna()
df1 = df[df.REVIEWBODY.notnull()]
df1 = df1.drop_duplicates(subset='REVIEWBODY', keep="first")
print(df1.shape)
df1.head()

(8618274, 10)


,NODE,URL,REVIEWBODY,REVIEWRATING,ITEMREVIEWED,REVIEWASPECT,REVIEWAUTHOR,REVIEWDESCR,BESTRATING,WORSTRATING
1,_:node1f865f4bfa138ac184ddbbd383299621,http://67kmm.com/~hello-neighbor/android,"""Hello Neighbor est un jeu d'horreur \u00C3\u0...",None,None,None,None,None,None,None
2,_:node7d8efbe93e9573eb609b41cfa3be84,http://67kmm.com/~snapchat/android,"""Bon7Snapchat est une application de partage i...",None,None,None,None,None,None,None
4,_:nodebb685880f9d030fca1b194c14548d8b5,http://67kmm.com/~grand-theft-auto-san-andreas...,"""Par Matt Mullarkey-TonerExcellent9Du haut de ...",None,None,None,_:node55c1dea05a3a6929edebabe6265ba3b,None,None,None
5,_:node43bf22a0b65d63fc2dd0eab5569c1eed,http://67kmm.com/~simcity-buildit/android,"""Par James ThorntonTr\u00C3\u00A8s bon8Que les...",None,None,None,_:node3c2baa505576d52de3618ffa81edfd,None,None,None
6,_:nodee181c617d55f8f7ebb8a6f0f277e8ad,http://angarsk.frf.name/page/8,"""\n\t\t\t\t\t\t\t\t\t\t\u0410\u043D\u0433\u043...",None,"""\u043E \u043A\u043E\u043C\u043F\u0430\u043D\u...",None,None,None,None,None


In [ ]:
def detectLang(row):
    try:
        return detect(row['REVIEWBODY'])
    except:
        print("exception:", row['REVIEWBODY'])
    else:
        print("sad:", row['REVIEWBODY'])
df1['LANGUAGE'] = df1.apply(detectLang, axis=1)

df_en = df1.loc[df1['LANGUAGE'] == "en"]

exception: "-"
exception: ":)"
exception: "."
exception: "++"
exception: ".."
exception: "\""
exception: "5"
exception: "<3"
exception: "38/4         "
exception: "555"
exception: "43383"
exception: "(: !"
exception: "+"
exception: ".   "
exception: "!!!"
exception: "****"
exception: "https://youtu.be/g8RCvnV-LPQ"
exception: "<3 "
exception: "\"5\""
exception: "01234567890"
exception: ":-)"
exception: "kk1117684@gamil.com"
exception: "\"4\""
exception: "........................... . . .  . . . . . . . ..  . . . . . . . . . .  . . . . . . . . . . .  . .    .  ."
exception: "http://angelguides.wordpress.com/2010/11..Thanks:)"
exception: "23/09/18"
exception: <https://mobiliteit.klantenvertellen.nl/beoordelingen/mak-auto-techniek/2121>
exception: "^^^^"
exception: ")))"
exception: ">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>"
exception: "(^^)"
exception: "*3.5/5"
exception: ":')"
exception: "???????????????????"
exception: ":'( :'("
exception: "69"
exception: "123"
exception: "!"
exception: "[..

In [ ]:
phone_lst = ['smartphone', 'phone',
             'phone case', 'phone cable', 'phone charger', 'phone mount',
             'cell phone', 'mobile phone']
contains = [df_en['a'].str.contains(i) for i in phone_lst]
result = df_en[np.all(contains, axis=0)]

# df_phone = df_en[df_en['REVIEWBODY'].isin(phone_lst)]
# print(df_phone.head())
# ANGUAGE'] == "en"]
df_en.head()